
---


# **PROJECT**

---


**Objective**
Create a popularity based recommender system at a genre level. User will
input a genre (g), minimum ratings threshold (t) for a movie and no. of
recommendations(N) for which it should be recommended top N movies
which are most popular within that genre (g) ordered by ratingsin descending
order where each movie has at least (t) reviews.

**Data Description**
The data consists of 105339 ratings applied to over 10329 movies. The average
rating is 3.5 and minimum and maximum rating is 0.5 and 5 respectively. There
are 668 users who have given their ratings for 149532 movies.

There are two data files which are provided:

***Movies.csv***

movieId: ID assigned to a movie.

title: Title of a movie

genres: pipe separated list of movie genres.

***Ratings.csv***

userId: ID assigned to a user

movieId: ID assigned to a movie

rating: rating by a user to a movie

Timestamp: time at which the rating was provided.

**Steps and Tasks**

Import libraries and load dataset

**Exploratory Data Analysis including:**
Understanding of distribution of the features available.
Finding unique users and movies.
Average rating and Total movies at genre level.
Unique genres considered.

Design the 3 different types of recommendation modules as mentioned in
objectives.
Additional/Optional: Create a GUI interface using Python libraries
(ipywidgets etc.) to play around with the recommendation modules.


In [ ]:
# Import libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load datasets
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Merge datasets for easier handling
movie_ratings = ratings.merge(movies, on='movieId')

# Create a combined genres column for content-based filtering
movies['combined_genres'] = movies['genres'].str.replace('|', ' ')

# Vectorize genres for content-based filtering
count_vectorizer = CountVectorizer()
genre_matrix = count_vectorizer.fit_transform(movies['combined_genres'])

# Compute cosine similarity matrix for content-based filtering
cosine_sim = cosine_similarity(genre_matrix, genre_matrix)

# Create user-item matrix for collaborative filtering
user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Compute user similarity for collaborative filtering
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)


# ----------------------------------------
# Step 1: Popularity-Based Recommender
# ----------------------------------------
def popularity_based_recommender(genre, min_reviews, top_n):
    genre_movies = movies[movies['genres'].str.contains(genre, case=False, na=False)]
    genre_ratings = movie_ratings[movie_ratings['movieId'].isin(genre_movies['movieId'])]

    # Group by title, calculate average rating and number of reviews
    popular_movies = genre_ratings.groupby('title').agg(
        average_rating=('rating', 'mean'),
        num_reviews=('rating', 'count')
    ).reset_index()

    # Filter by minimum reviews and sort by average rating
    popular_movies = popular_movies[popular_movies['num_reviews'] >= min_reviews]
    popular_movies = popular_movies.sort_values(by='average_rating', ascending=False).head(top_n)

    return popular_movies[['title', 'average_rating', 'num_reviews']]

# ----------------------------------------
# Step 2: Content-Based Recommender
# ----------------------------------------
def content_based_recommender(movie_title, top_n=5):
    try:
        movie_idx = movies[movies['title'].str.lower() == movie_title.lower()].index[0]
    except IndexError:
        return "Movie not found in the dataset."

    sim_scores = list(enumerate(cosine_sim[movie_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]  # Exclude the movie itself

    movie_indices = [i[0] for i in sim_scores]
    recommended_movies = movies.iloc[movie_indices][['title']]
    return recommended_movies.reset_index(drop=True)

# ----------------------------------------
# Step 3: Collaborative Filtering Recommender
# ----------------------------------------
def collaborative_based_recommender(user_id, k=10, top_n=5):
    if user_id not in user_similarity_df.index:
        return "User not found in the dataset."

    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:k+1].index
    similar_users_ratings = user_movie_matrix.loc[similar_users].mean(axis=0)

    # Filter out movies already rated by the user
    user_rated_movies = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id] > 0].index
    recommendations = similar_users_ratings.drop(index=user_rated_movies).sort_values(ascending=False).head(top_n)

    recommended_movies = movies[movies['movieId'].isin(recommendations.index)][['title']]
    return recommended_movies.reset_index(drop=True)

# ----------------------------------------
# Interactive GUI with ipywidgets
# ----------------------------------------

# Widgets for Popularity-Based Recommender
genre_input = widgets.Text(description="Genre:")
min_reviews_input = widgets.IntText(description="Min Reviews:")
top_n_input = widgets.IntText(description="Top N:")
popularity_button = widgets.Button(description="Get Recommendations")
popularity_output = widgets.Output()

# Callback for Popularity-Based Recommender
def on_popularity_button_click(b):
    with popularity_output:
        clear_output()
        genre = genre_input.value
        min_reviews = min_reviews_input.value
        top_n = top_n_input.value
        try:
            recommendations = popularity_based_recommender(genre, min_reviews, top_n)
            display(recommendations)
        except Exception as e:
            display(f"Error: {e}")

popularity_button.on_click(on_popularity_button_click)

# Widgets for Content-Based Recommender
movie_input = widgets.Text(description="Movie Title:")
content_top_n_input = widgets.IntText(description="Top N:")
content_button = widgets.Button(description="Get Similar Movies")
content_output = widgets.Output()

# Callback for Content-Based Recommender
def on_content_button_click(b):
    with content_output:
        clear_output()
        movie_title = movie_input.value
        top_n = content_top_n_input.value
        try:
            recommendations = content_based_recommender(movie_title, top_n)
            display(recommendations)
        except Exception as e:
            display(f"Error: {e}")

content_button.on_click(on_content_button_click)

# Widgets for Collaborative-Based Recommender
user_id_input = widgets.IntText(description="User ID:")
similar_users_input = widgets.IntText(description="K Users:")
collab_top_n_input = widgets.IntText(description="Top N:")
collab_button = widgets.Button(description="Get Recommendations")
collab_output = widgets.Output()

# Callback for Collaborative-Based Recommender
def on_collab_button_click(b):
    with collab_output:
        clear_output()
        user_id = user_id_input.value
        k_users = similar_users_input.value
        top_n = collab_top_n_input.value
        try:
            recommendations = collaborative_based_recommender(user_id, k_users, top_n)
            display(recommendations)
        except Exception as e:
            display(f"Error: {e}")

collab_button.on_click(on_collab_button_click)

# Display Widgets
print("Popularity-Based Recommender:")
display(genre_input, min_reviews_input, top_n_input, popularity_button, popularity_output)

print("\nContent-Based Recommender:")
display(movie_input, content_top_n_input, content_button, content_output)

print("\nCollaborative-Based Recommender:")
display(user_id_input, similar_users_input, collab_top_n_input, collab_button, collab_output)


Popularity-Based Recommender:


Text(value='', description='Genre:')

IntText(value=0, description='Min Reviews:')

IntText(value=0, description='Top N:')

Button(description='Get Recommendations', style=ButtonStyle())

...

IntText(value=0, description='User ID:')

IntText(value=0, description='K Users:')

IntText(value=0, description='Top N:')

Button(description='Get Recommendations', style=ButtonStyle())

Output()